<a href="https://colab.research.google.com/github/david132313/DataManagment_Source/blob/main/1_7_Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Decorators: Used to add extra features to a function

## *args

* we used *args in a function definition to specify that a function can take an arbitrary number of arguments represented as a tuple
* *args goes in the parameter list
* args goes in the function body to represent the tuple of arguments


## An example of *args

In [ ]:
def my_func(*args):
  return (max(args),min(args))

print(my_func(1,2,3))
print(my_func(1,2,3,4,-2))

(3, 1)
(4, -2)


This can be used more generally. * will unpack a tuple so it can be used in a function call, for example:

In [ ]:
t=(10,0,-2)
for i in range(*t):
 print(i)

10
8
6
4
2


Will the following work?

In [ ]:
t=(10,0,-2)
try:
  for i in range(t):
    print(i)
except TypeError as e:
    print("TypeError",e)

TypeError 'tuple' object cannot be interpreted as an integer


What is the difference between the following two function calls?

In [ ]:
def f(*args):
  print(args)

def g(*args):
  print(*args)

f(1,2)
g(1,2)

(1, 2)
1 2


In Python, functions are first-class objects, which means they can be passed as arguments, returned as results from other functions, stored in data structures, etc. Decorators rely on this fact.

Here's an example where we pass a function name as a parameter to another function and the function passed in is then executed.

In [ ]:
def double(n):
  return n*2
def square(n):
  return n*n
def do_something(func,n):
  return func(n)
print(do_something(double,3))
print(do_something(square,3))
print(type(double))

6
9
<class 'function'>


##Exercise:

Define the two functions above, double, and square, using lambda instead of def.

In [ ]:
#@title Solution


An aside: you can even use "eval" to convert a string into something that can be evaluated. But this is dangerous to use with user input because it sometimes can be used to run arbitrary operating system commands.

In [ ]:
s2=eval("square")
print(s2(5))
print(eval("double(14)"))


25
28


Here's an example where we store function names as values in a dictionary and then call the functions by referencing the values.

In [ ]:
def add(*args):
  r=0
  for x in args:
    r+=x
  return r

def product(*args):
  r=1
  for x in args:
    r*=x
  return r

print(add(1,2,3,4))
print(product(1,2,3,4))
d={"add":add, "product":product}
print(d["add"](1,2,3,4,5))
print(d["product"](1,2,3,4,5))

10
24
15
120


##Exercise:

Rewrite the function product above using lambda and reduce. (Note reduce needs to be imported from the functools module).

In [ ]:
from functools import reduce
def product(*args):
  return reduce(lambda x,y: x*y, args)

print(product(1,2,3))


6


Python also supports "inner functions" which are local to the function in which they are defined. They cannot be called from outside that function, like a local variable cannot be referenced outside the function it is defined in.

In [ ]:
def greeter():
  # these functions are local to the function
  # they are wrapped in
  def hi():
    print("hi")
  def hello():
    print("hello")
  hello()
  hi()

greeter()

hello
hi


Can we do the following?

In [ ]:
try:
  hi()
except NameError as e:
  print("NameError",e)

NameError name 'hi' is not defined


The following function runMany returns the function new_func as its result. The function new_func is itself defined as an inner function in terms of the arguments to runMany and takes one argument of its own.

In [ ]:
def runMany(func,n):
  # what is x? the argument to the function returned
  def new_func(x):
    for i in range(n):
      x=func(x)
    return(x)
  return new_func


When runMany is called with the arguments double (which is a function) and 3, it returns a new function which doubles its argument 3 times, or octuples it (that is, makes it eight times as large). Thus we can use functions to create new functions that are based on them. This is the idea behind decorators.

In [ ]:
octuple=runMany(double,3)
print(type(octuple))
print(octuple(5))

<class 'function'>
40


In what follows, we have a cake, or rather a function called cake which returns the word cake.

In [ ]:
def cake():
  return "cake"

print(cake())

cake


Let's decorate it. The new cake contains icing.

In [ ]:
def icing(old_func):
  def new_func():
    return old_func()+' with icing'
  return new_func

#def cake(): return "cake"
#cake=icing(cake)

@icing
def cake(): return "cake"

print(cake())

cake with icing


In [ ]:
def icing(old_func):
  def new_func():
    return old_func()+' with icing'
  return new_func

def cake(): return "cake"

new=icing(cake)
new()

'cake with icing'

The following is another, equivalent way to add the decorator. It is a form of syntactic sugar.

In [ ]:
@icing
def cake():
  return "cake"
print(cake())

cake with icing


We could decorate a different function.

In [ ]:
@icing
def cupcake():
  return "cupcake"
print(cupcake())

cupcake with icing


**Poll** **Question**:

The function that is returned by a decorator is a type of function that is called:

* An embedded function
* An inner or nested function
* A local function
* A private function


Let's make a decorator that adds exclamation points. (Note what happens if we run the following cell multiple times.)

In [ ]:
def shout(old_f):
  def new_f(s):
    return old_f(s)+'!!!!!'
  return new_f

# decorate the str function
str=shout(str)
print(str(12))

12!!!!!


Does this work?

In [ ]:
@shout
def full_name(first,last):
  return f"{first} {last}"

try:
  person=full_name('Alice','Alvarez')
  print(person)
except TypeError as e:
  print("TypeError:",e)

TypeError: shout.<locals>.new_f() takes 1 positional argument but 2 were given


We can fix this by allowing for a variable amount of arguments in the decorator.

In [ ]:
def shout(old_f):
    def new_f(*args, **kwargs):
        res = old_f(*args)
        return res + '!!!!!'
    return new_f

In [ ]:
@shout
def full_name(first,last):
  return f"{first} {last}"

person=full_name('Alice','Alvarez')
print(person)

Alice Alvarez!!!!!


## Uses of Decorators
* Modify arguments going into a function, or modify the function's output
* Do some things before or after we call a function
* Some examples
    - timing a function
    - caching results to speed up a function

In [ ]:
import time
def time_ns(func):
  def wrapper(*arg):
      t = time.process_time_ns()
      res = func(*arg)
      print(f"The function {func.__name__} took {time.process_time_ns()-t} nanoseconds to run.")
      return res
  return wrapper

# series starts with 0,1,1,2,3,5 ...
@time_ns
def fibonacci(n):
  l=[0,1]
  for i in range(2,n):
    l.append(l[i-1]+l[i-2])
  return l[n-1]

print(fibonacci(6))
print(fibonacci(20))
print(fibonacci(100))


The function fibonacci took 10688 nanoseconds to run.
5
The function fibonacci took 11385 nanoseconds to run.
4181
The function fibonacci took 21338 nanoseconds to run.
218922995834555169026


Decorators can be used for caching. Here the lru_cache decorator saves up to the last maxsize values returned from calls to the function decorated. If maxsize is None, it caches all of the previous calls.

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print([fib(n) for n in range(30)])

print(fib.cache_info())

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229]
CacheInfo(hits=56, misses=30, maxsize=None, currsize=30)


Usually we don't write our own decorators, but use pre-written ones (like in the two examples above.) For instance:

* Python uses a decorator to define a method as [static](https://docs.python.org/3/howto/descriptor.html?highlight=static%20method#static-methods).
* The flask web framework uses decorators for http request handling.
